In [2]:
import pandas as pd
import datetime as dt

In [16]:
#reading in CSV, assigning to 'stock_data' and checking dataframe

stock_data = pd.read_csv('./stock_data/stocks_latest/stock_prices_latest.csv')
stock_data.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


In [17]:
#Converting date to datetime then filtering down just to stock prices from 2016 on

stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data_trim = stock_data[(stock_data['date'] > '2015-12-31')]
stock_data_trim.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.750,51.83,49.7013,20032017,1.0
8,MSFT,2017-05-08,68.97,69.05,68.420,68.94,67.8482,18446053,1.0
11,MSFT,2016-03-24,53.84,54.33,53.730,54.21,51.9835,19949972,1.0
22,MSFT,2017-12-27,85.65,85.98,85.215,85.71,85.7100,13000828,1.0
24,MSFT,2017-05-01,68.68,69.55,68.500,69.41,68.3107,31304672,1.0


In [31]:
#confirming that the new dataframe is smaller than the old
print(stock_data.shape)
print(stock_data_trim.shape)

(23528435, 9)
(7650832, 9)


In [25]:
#reading in company descriptions, assigning to 'company_data' and checking dataframe
company_data = pd.read_csv('./S&P500-Info_Twitter_link.csv')

#dropping unnamed column and checking dataframe
company_data = company_data.drop(['Unnamed: 0'], axis =1)
company_data.head()

,symbol,security,sec filings,gics sector,gics sub-industry,headquarters location,date first added,cik,founded,twitter handle
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",8/9/1976,66740,1902,3MNews
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",3/31/1964,1800,1888,abbottglobal
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",12/31/2012,1551152,2013 (1888),AbbvieUS
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",5/31/2018,815094,1981,Abiomedimpella
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",7/6/2011,1467373,1989,Accenture


In [35]:
#merging stock and company description dataframes, only keeping S&P 500 company stock data
company_stock_data = stock_data_trim.merge(company_data, on = 'symbol')

#checking to make sure that all others have been removed
company_stock_data['security'].isna().sum()

0

In [37]:
#Exporting trimmed file
company_stock_data.to_csv(r'\stock_prices_trimmed.csv', index = False)